Here is the link to the Github Repository: https://github.com/fhawkings/QT2005_Assessment-3_21010680514 https://github.com/fhawkings/2C_21010680514

In [ ]:
# Importing all of the relevant libraries.

import pandas as pd
import os
import requests
import re
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
!pip install GeoText
from geotext import GeoText
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from collections import Counter
import matplotlib.pyplot as plt
import spacy
from collections import defaultdict
import numpy as np
import plotly.graph_objects as go

In [ ]:
# Setting the Client ID and the Secret Key from the Reddit API dashboard.

CLIENT_ID = 'woKdnKEcjkuZRQGeYo7d4w'
SECRET_KEY = 's9a8JGK-pydiDp69EGqG3HgjpBxGjw'

# Requesting the authorisation.

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

# Creating a dictionary which is containing the necessary data and password access for my token request.

data = {
    'grant_type': 'password',
    'username': 'Felix_Hawkings',
    'password': 'kyCtof-9josco-rozsog'
}

# Identifying version of the API.

headers = {'User-Agent': 'MyAPI/0.0.1'}

# Send a POST request to obtain the access to the API using login data, auth and headers to give the access token.

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# Extract the access token from the response JSON and store it in a variable.

TOKEN = res.json()['access_token']

In [ ]:
# Adding the access token into the headers to allow access while applying analysis.

headers['Authorization'] = f'bearer {TOKEN}'

In [ ]:
# Sending a GET request to the specified URL with headers to provide access, then retrieving the response and parse it as JSON.

requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

In [ ]:
# Sending a GET request to test the specified URL with the subreddit of ubrban planning with headers to provide access, then retrieving the response and parse it as JSON.

res = requests.get('https://oauth.reddit.com/r/urbanplanning/hot',
                   headers=headers)


In [ ]:
# Defining the subreddit names and saving them to a variable.

subreddits = ['urbanplanning', 'PlaceMaking', 'Urbanism','parks', 'coffeeshops',
              'libraries', 'communitygardening', 'hiking', 'CulturalHeritage', 'Anthropology',
              'Neighborhoods', 'PublicSpaces']

# Initializing and creating an empty DataFrame and saving it to df.

df = pd.DataFrame(columns=['subreddit', 'title', 'selftext'])

# Making a GET requests for each subreddit under the variable above and extracting the data.

for subreddit in subreddits:
    url = f'https://oauth.reddit.com/r/{subreddit}/hot'
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'User-Agent': 'MyAPI/0.0.1'
    }
    response = requests.get(url, headers=headers)

# Processing the response and pulling out the children variable which is a list that contains all the data that I need to extract.
    
    if response.status_code == 200:
        data = response.json()
        children = data['data']['children']
        
# Extracting the data that is relevent and appending it to the DataFrame as well as adding in a failsafe if this fails then it will print a specific line which helps with troubleshooting.

        for child in children:
            title = child['data']['title']
            selftext = child['data']['selftext']
            df = df.append({'subreddit': subreddit, 'title': title, 'selftext': selftext}, ignore_index=True)
    else:
        print(f"Error retrieving data for subreddit '{subreddit}':", response.status_code)

print(df)


In [ ]:
# Downloading NLTK resources that are required to do the analysis in this code block.

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initializing both lemmatizer and stopwords.

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function below to clean and lemmatize all the text.

def clean_text(text):
    text = re.sub(r'\d+', '', text)  # Removing all the numbers.
    text = re.sub(r'\W+', ' ', text)  # Removing all the symbols.

# Lemmatize, remove stopwords, and tokenize all the words in the text column.

    cleaned_text = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text) if word.isalnum() and word.lower() not in stop_words])
    return cleaned_text

# Doing the exact same but to the selftext column.

df['selftext'] = df['selftext'].apply(clean_text)


print(df['selftext'])

In [ ]:
# Loading in the spacy model.

nlp = spacy.load('en_core_web_sm')

# Defining a list of all the public locations that I want to understand if they are in the main dataframe.

target_locations = ['Neighbourhood', 'Park', 'Recreation Area', 'Community Center', 'Library',
                    'Places of Worship', 'School', 'tennis court', 'Cultural Center','Museum',
                    'CoffeeShop','Cafe', 'Local Business', 'Public Transportation Hub','Local Business', 'Public Transportation Hub','Beache', 'Playground', 'Community Center','Restaurant', 
                    'Shopping Mall', 'Amusement Park', 'Stadium', 'Concert Hall', 'Theater', 'Gym', 'Fitness Center', 
                    'Public Plaza', 'Outdoor Market', 'Picnic Area', 'Recreation Center', 
                    'Conference Venue', 'Art Gallery', 'Museum', 'Garden', 'Hiking Trail', 'Sports Field', 
                    'Swimming Pool', 'Skate Park', 'Public BBQ Area', 'Historical Site', 
                    'Street Festival']

# Creating a dictionary to store all the location as counts.

location_counts = defaultdict(int)

# Iterating through each row in the DataFrame.

for index, row in df.iterrows():
    text = row['selftext']
    doc = nlp(text)

# Tokenize all the text and then check if any of the public locations are present.

    tokens = [token.text.lower() for token in doc]
    for location in target_locations:
        if location.lower() in tokens:
            location_counts[location] += 1

# Plot a bar chart of the most common locations and if there are no locations found then print an error message to speed up troubleshooting.

top_locations = sorted(location_counts.items(), key=lambda x: x[1], reverse=True)
if top_locations:
    location_labels, location_values = zip(*top_locations)

    plt.figure(figsize=(12, 8))
    plt.bar(location_labels, location_values)
    plt.title('Most Common Locations')
    plt.xlabel('Location')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()
else:
    print('No common locations found.')


In [ ]:
# Loading in the spacy model.

nlp = spacy.load('en_core_web_sm')

# # Defining a list of all the public locations that I want to understand if they are in the main dataframe.

target_locations = ['Neighbourhood', 'Park', 'Recreation Area', 'Community Center', 'Library',
                    'Places of Worship', 'School', 'tennis court', 'Cultural Center', 'Museum',
                    'CoffeeShop', 'Cafe', 'Coffee Shop', 'Local Business', 'Public Transportation Hub', 'Local Business',
                    'Public Transportation Hub', 'Playground', 'Community Center', 'Restaurant',
                    'Shopping Mall', 'Amusement Park', 'Stadium', 'Concert Hall', 'Theater', 'Gym', 'Fitness Center',
                    'Public Plaza', 'Outdoor Market', 'Picnic Area', 'Recreation Center',
                    'Conference Venue', 'Art Gallery', 'Museum', 'Garden', 'Sports Field',
                    'Swimming Pool', 'Skate Park', 'Public BBQ Area', 'Historical Site',
                    'Street Festival']

# Creating a dictionary to store all the location as counts.

location_counts = defaultdict(int)

# Iterating through each row in the DataFrame.

for index, row in df.iterrows():
    text = row['selftext']
    doc = nlp(text)

# Tokenize all the text and then check if any of the public locations are present.

    tokens = [token.text.lower() for token in doc]
    for location in target_locations:
        if location.lower() in tokens:
            location_counts[location] += 1

# Sorting the location counts and saving that to a variable.

top_locations = sorted(location_counts.items(), key=lambda x: x[1], reverse=True)

if top_locations:
    location_labels, location_values = zip(*top_locations)

# So then each bar is a different colour I have used this code to apply custom colors for each bar.

    colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
              'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
              'rgb(188, 189, 34)', 'rgb(23, 190, 207)', 'rgb(174, 199, 232)', 'rgb(255, 152, 150)',
              'rgb(152, 223, 138)', 'rgb(255, 187, 120)', 'rgb(197, 176, 213)', 'rgb(196, 156, 148)',
              'rgb(247, 182, 210)', 'rgb(219, 219, 141)', 'rgb(158, 218, 229)', 'rgb(23, 190, 207)',
              'rgb(188, 189, 34)', 'rgb(23, 190, 207)', 'rgb(214, 39, 40)', 'rgb(255, 127, 14)',
              'rgb(140, 86, 75)', 'rgb(31, 119, 180)', 'rgb(148, 103, 189)', 'rgb(227, 119, 194)',
              'rgb(127, 127, 127)', 'rgb(188, 189, 34)', 'rgb(152, 223, 138)', 'rgb(174, 199, 232)',
              'rgb(255, 187, 120)', 'rgb(197, 176, 213)', 'rgb(23, 190, 207)', 'rgb(196, 156, 148)',
              'rgb(219, 219, 141)', 'rgb(247, 182, 210)', 'rgb(158, 218, 229)', 'rgb(23, 190, 207)',
              'rgb(188, 189, 34)', 'rgb(214, 39, 40)']

# Creating a bar chart using Plotly to make it easier to see and understand. 

    fig = go.Figure(data=[
        go.Bar(
            x=location_labels,
            y=location_values,
            marker=dict(color=colors)
        )
    ])

# Seting the title and labelling the axis's.

    fig.update_layout(
        title='Most Common Locations',
        xaxis_title='Location',
        yaxis_title='Count'
    )

# Rotating the x-axis labels for better readability due to some location names being long.

    fig.update_xaxes(tickangle=45)

# Displaying the chart and putting a else print error message to help with troubleshooting.
    
    fig.show()
else:
    print('No common locations found.')


In [ ]:
# Loading the VAD scores from the Warriner rescale which is loacted under the filepath.

vad = pd.read_csv('/filepathname/Warriner_rescale.csv', index_col=0)
vad = vad[['V.Mean.Sum', 'A.Mean.Sum', 'D.Mean.Sum']]
vad.columns = ['valence', 'arousal', 'dominance']

# Defining a function to get the specific VAD scores for the locations.

def get_vad_scores(word):
    if word.lower() in vad.index:
        scores = vad.loc[word.lower()].values
        return scores
    else:
        return [np.nan, np.nan, np.nan]
    
# Creating a new column in the dataframe to store the new VAD scores.

df['vad_scores'] = [[] for _ in range(len(df))]

In [ ]:
# Defining a dictionary to store the location VAD scores.

location_vad_scores = {}

# Applying VAD analysis to the top four locations.

for location in top_locations[:5]:
    location_name = location[0]
    vad_scores = get_vad_scores(location_name)
    location_vad_scores[location_name] = vad_scores

# Printing the VAD scores for the top four locations.

for location, vad_scores in location_vad_scores.items():
    print(f"Location: {location}")
    print(f"Valence: {vad_scores[0]}")
    print(f"Arousal: {vad_scores[1]}")
    print(f"Dominance: {vad_scores[2]}")
    print()


In [ ]:
# Extracting the VAD scores and names for each location.

locations = list(location_vad_scores.keys())
valence = [vad_scores[0] for vad_scores in location_vad_scores.values()]
arousal = [vad_scores[1] for vad_scores in location_vad_scores.values()]
dominance = [vad_scores[2] for vad_scores in location_vad_scores.values()]
names = locations  

# Creating a trace to use the scatter plot.

trace = go.Scatter3d(
    x=valence,
    y=arousal,
    z=dominance,
    mode='markers',
    marker=dict(
        size=5,
        color=valence,  
        colorscale='Viridis',  
        opacity=0.8
    ),
    name='VAD Score',  # Name if the plot.
    text=names, 
    hovertemplate='Name: %{text}<br>Valence: %{x}<br>Arousal: %{y}<br>Dominance: %{z}'  # Hover over template code for the VAD Results.
)

# Creating the layout for the plot.

layout = go.Layout(
    scene=dict(
        xaxis=dict(title='Valence'),
        yaxis=dict(title='Arousal'),
        zaxis=dict(title='Dominance')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Creating the figure and then adding the trace and layout.

fig = go.Figure(data=[trace], layout=layout)

# Displaying the plot with the specified name.

fig.update_layout(
    title='VAD Scores for Locations'  
)

fig.show()


In [ ]:
# Exporting to CSV file to save the dataframe.

df.to_csv('filepathname', index=False)